## wczytanie danych i wstępna  analiza

In [1]:
# najczesciej wystepujace slowo 

In [54]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
import nltk
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
from time import time
import logging
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.linear_model import  LogisticRegressionCV
from sklearn.decomposition import NMF
from nltk.stem import WordNetLemmatizer
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer,MaxAbsScaler
from sklearn.cluster import KMeans
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ania\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## wczytanie danych i wstępna analiza

In [69]:
df = pd.read_csv("train.tsv", sep="\t")
df
df['klasa']= df.label.apply(lambda x: 1 if x == 'pants-fire' else 0  )
df = df.drop(columns =['label'])

In [70]:
pd.value_counts(df['klasa'].values, sort=False)

0    9426
1     842
dtype: int64

In [71]:
df.columns

Index(['statement', 'subject', 'speaker', 'speaker_job', 'state', 'party',
       'context', 'klasa'],
      dtype='object')

In [72]:
df.groupby('klasa').describe().T

klasa                                                               0  \
statement   count                                                9426   
            unique                                               9409   
            top     On changing the rules for filibusters on presi...   
            freq                                                    3   
subject     count                                                9426   
            unique                                               3588   
            top                                           health-care   
            freq                                                  345   
speaker     count                                                9426   
            unique                                               2759   
            top                                          barack-obama   
            freq                                                  485   
speaker_job count                                                6862   
            unique                                               1141   
            top                                             President   
            freq                                                  489   
state       count                                                7533   
            unique                                                 81   
            top                                               Florida   
            freq                                                  956   
party       count                                                9426   
            unique                                                 23   
            top                                            republican   
            freq                                                 4129   
context     count                                                9337   
            unique                                               4043   
            top                                        a news release   
            freq                                                  220   

klasa                                                               1  
statement   count                                                 842  
            unique                                                842  
            top     Democrat Russ Feingold announced his Wisconsin...  
            freq                                                    1  
subject     count                                                 842  
            unique                                                509  
            top                                           health-care  
            freq                                                   37  
speaker     count                                                 842  
            unique                                                428  
            top                                           chain-email  
            freq                                                   82  
speaker_job count                                                 505  
            unique                                                205  
            top                                       President-Elect  
            freq                                                   52  
state       count                                                 526  
            unique                                                 46  
            top                                                 Texas  
            freq                                                   87  
party       count                                                 842  
            unique                                                 14  
            top                                            republican  
            freq                                                  380  
context     count                                                 831  
          

In [73]:
countV = CountVectorizer(analyzer='word', binary=True, decode_error='strict',  input='content',
    lowercase=True, max_df=0.5, max_features=None, min_df=1,
    ngram_range=(1, 1), preprocessor=None, stop_words='english',
    strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
    tokenizer=None, vocabulary=None)

In [74]:
class StemmingVectorizer(TfidfVectorizer):            
    def build_tokenizer(self):                
        self.wnl = WordNetLemmatizer()
        tokenize = super().build_tokenizer()
        return lambda doc: [self.wnl.lemmatize(w)
                for w in tokenize(doc)]

In [75]:
pipeline = Pipeline([
    ('tfidf', StemmingVectorizer()),
    ('scaler', MaxAbsScaler()),
    #('reduce_dim',  NMF()),
    ('lr', LogisticRegression(penalty = 'l2', C=1.0))
])

In [76]:
y=df['klasa']
X = df.drop(columns= ['klasa'])['statement']

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [78]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 StemmingVectorizer(analyzer='word', binary=False,
                                    decode_error='strict',
                                    dtype=<class 'numpy.float64'>,
                                    encoding='utf-8', input='content',
                                    lowercase=True, max_df=1.0,
                                    max_features=None, min_df=1,
                                    ngram_range=(1, 1), norm='l2',
                                    preprocessor=None, smooth_idf=True,
                                    stop_words=None, strip_accents=None,
                                    sublinear_tf=False,
                                    token_patter...\w\\w+\\b',
                                    tokenizer=None, use_idf=True,
                                    vocabulary=None)),
                ('scaler', MaxAbsScaler(copy=True)),
                ('lr',
                 LogisticRegres

In [79]:
cross_val_score(pipeline, X_train, y_train, scoring="roc_auc", cv=10).mean()


0.6647669228485051

In [80]:
parameters = {
    'lr_C' : [0.1, 1, 5, 10],
    'lr_penalty' : ['l1', 'l2'],
    'reduce_dim__n_components': [10, 50, 100],
    #'reduce_dim': [TruncatedSVD(), NMF()],
    
}

## zbior testowy

In [83]:
df_test = pd.read_csv("test_noy.tsv", sep="\t")


In [84]:
df_test = df_test['statement']

In [85]:
pipeline.fit(X, y)

Pipeline(memory=None,
         steps=[('tfidf',
                 StemmingVectorizer(analyzer='word', binary=False,
                                    decode_error='strict',
                                    dtype=<class 'numpy.float64'>,
                                    encoding='utf-8', input='content',
                                    lowercase=True, max_df=1.0,
                                    max_features=None, min_df=1,
                                    ngram_range=(1, 1), norm='l2',
                                    preprocessor=None, smooth_idf=True,
                                    stop_words=None, strip_accents=None,
                                    sublinear_tf=False,
                                    token_patter...\w\\w+\\b',
                                    tokenizer=None, use_idf=True,
                                    vocabulary=None)),
                ('scaler', MaxAbsScaler(copy=True)),
                ('lr',
                 LogisticRegres

In [104]:
pwstwa = pipeline.predict_proba(df_test)[:,1] 

In [105]:
pwstwa = pd.DataFrame({'Anna Kozioł' : pwstwa})

In [107]:
pwstwa.to_csv('scory_text_mining.res', sep='\t')

In [106]:
pwstwa

,Anna Kozioł
0,0.063178
1,0.208810
2,0.063505
3,0.058048
4,0.020030
...,...
2560,0.002008
2561,0.013123
2562,0.056423
2563,0.025769
